In [9]:
pip install qdrant-client

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [10]:
from qdrant_client import QdrantClient
client = QdrantClient(host="localhost", port=6333)

In [11]:
print('HTTP', requests.get("http://localhost:6333/healthz").text)

HTTP healthz check passed


In [12]:
import requests
r = requests.get(
    "http://localhost:6333/healthz",
    proxies={"http": None, "https": None}  # обход всех proxy
)
print(r.text)          # → {"status":"ok"}

healthz check passed


In [13]:
from qdrant_client import QdrantClient
client = QdrantClient(url="http://localhost:6333")
print(client.get_collections())

collections=[CollectionDescription(name='articles')]


In [14]:
from qdrant_client.http import models as rest
# Размерность вектора (size) - задается один раз и ее нельзя менять 
client.recreate_collection(
    collection_name="articles",
    vectors_config=rest.VectorParams(size=384, distance=rest.Distance.COSINE)
)

/var/folders/22/w_0gdhgj699dgr2cp6bwhn800000gn/T/ipykernel_6788/3968867832.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

Поэтому во многих базах данных (Qdrant, Weaviate и др.) при создании коллекции с метрикой Cosine векторы автоматически нормализуются при сохранении. 
HNSW-индекс: идет снизу вверх и тщательно перебирает все значения. 
Во время поиска алгоритм идёт жадно: на каждом шаге переходит к соседу, чей вектор ближе к запросу. Если дальше улучшений нет — спускается уровнем ниже, пока не дойдёт до самого подробного слоя. 

## **Qdrant (“Quadrant”)** — это специализированная база данных для работы с векторными представлениями (vector embeddings). Коротко: она хранит и быстро ищет объекты, описанные не привычными строками/числами, а много-мерными векторами.

In [15]:
from sentence_transformers import SentenceTransformer
import numpy as np
#query -> применяем к ним vector 
sentences = [
    "Кот играет на ковре",      # id = 0
    "Собака тихо спит",         # id = 1
    "Автомобиль быстро едет",   # id = 2
    "Инвестор закупает акции",
    "Котенок играет с клубком",
    "Корова жует траву" # id = 2
]
model = SentenceTransformer("all-MiniLM-L6-v2")

vectors = model.encode(sentences, normalize_embeddings=True)

In [24]:
# Каждая точка = id + vector + payload. Поля из payload можно использовать в фильтрах.
payloads = [                         # любое поле можно фильтровать
    {"tag": "animals",  "year": 2023},
    {"tag": "animals",  "year": 2022},
    {"tag": "vehicles", "year": 2023},
    {"tag": "people",   "year": 2021},
    {"tag": "animals",  "year": 2024},
    {"tag": "animals",  "year": 2023},
]

client.upsert(
    collection_name="articles",
    points=[
        rest.PointStruct(id=i, vector=v.tolist(), payload=payloads[i])
        for i, v in enumerate(vectors)
    ]
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [40]:
query_vec = model.encode(
    "котенок жует траву", normalize_embeddings=True
) 

hits = client.search(
    collection_name="articles",
    query_vector=query_vec.tolist(),
    limit=4
)

for hit in hits:
    print(f"score={hit.score:.3f}  →  {sentences[hit.id]}")

score=0.903  →  Корова жует траву
score=0.732  →  Котенок играет с клубком
score=0.702  →  Кот играет на ковре
score=0.611  →  Инвестор закупает акции


/var/folders/22/w_0gdhgj699dgr2cp6bwhn800000gn/T/ipykernel_6788/2478660548.py:5: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(


Поиск в Qdrant: базовый k-NN и поиск с фильтрацией 

In [19]:
#2.1 Базовый поиск (только по вектору)
from qdrant_client import QdrantClient, models

client = QdrantClient(host="localhost", port=6333)

In [21]:
query_vector = [0.12]*384 ## ← эмбеддинг запроса
top_k = 5

hits = client.search(
    collection_name = "articles",
    query_vector=query_vector,
    limit=top_k,
    with_vectors=False,              # в ответ не тащим сами векторы
    with_payload=True                # но хотим метаданные
)

for hit in hits:
    print(f"ID={hit.id:>4}  score={hit.score:.4f}  category={hit.payload.get('category')}")

ID=   1  score=0.0086  category=None
ID=   5  score=-0.0065  category=None
ID=   0  score=-0.0076  category=None
ID=   2  score=-0.0142  category=None
ID=   4  score=-0.0167  category=None


/var/folders/22/w_0gdhgj699dgr2cp6bwhn800000gn/T/ipykernel_6788/3525416721.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(


1. should=[…] — OR-условия;
2. must_not=[…] — исключить записи;
3. range, geo_bounding_box, geo_radius — числовые диапазоны и поиск по координатам.

In [36]:
#Поиск с фильтром по payload
from datetime import datetime

flt = models.Filter(
    must=[
        models.FieldCondition(
            key="tag",
            match=models.MatchValue(value="animals")
        ),
        models.FieldCondition(
            key="year",
            range=models.Range(gte=2023)
        )
    ]
)

hits = client.search(
    collection_name="articles",
    query_vector=query_vector,
    limit=top_k,
    query_filter=flt,
    with_payload=True
)

/var/folders/22/w_0gdhgj699dgr2cp6bwhn800000gn/T/ipykernel_6788/3088815193.py:17: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(


In [37]:
for hit in hits:
    print(f"ID={hit.id:>4}  score={hit.score:.4f}  category={hit.payload.get('tag')}")

ID=   5  score=-0.0065  category=animals
ID=   0  score=-0.0076  category=animals
ID=   4  score=-0.0167  category=animals


In [38]:
hits = client.search(
    collection_name="articles",
    query_vector=query_vector,
    limit=top_k,
    search_params=models.SearchParams(
        hnsw_ef=256,   # ↑ точность, ↓ скорость (обычный диапазон 32‒256)
        exact=False    # False = Approximate Nearest Neighbour (по умолчанию)
    )
)

for hit in hits:
    print(f"ID={hit.id:>4}  score={hit.score:.4f}  category={hit.payload.get('tag')}")

ID=   1  score=0.0086  category=animals
ID=   5  score=-0.0065  category=animals
ID=   0  score=-0.0076  category=animals
ID=   2  score=-0.0142  category=vehicles
ID=   4  score=-0.0167  category=animals


/var/folders/22/w_0gdhgj699dgr2cp6bwhn800000gn/T/ipykernel_6788/1205534079.py:1: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(


## Массовая загрузка данных и базовая статистика коллекции

In [42]:
import numpy as np
from qdrant_client import QdrantClient, models

client = QdrantClient("http://localhost:6333")
# Сгенерируем пример данных
N          = 20_000
vectors    = np.random.rand(N, 768).astype(np.float32)          # (N, 768)
categories = np.random.choice(["tech", "finance", "sports"], N)
years      = np.random.randint(2021, 2025, N)

In [43]:
points = [ #client.upload_points автоматически разобьёт массив на батчи.
    models.PointStruct(
        id=i,
        vector=vectors[i],
        payload={
            "category": str(categories[i]),
            "year": int(years[i])
        }
    ) for i in range(N)
]

client.upload_points(
    collection_name="articles",
    points=points,
    batch_size=1000,         # ← задаём желаемый размер пакета
    parallel=4               # ← потоков (CPU-ядра); 0 = авто
)

print("Импорт завершён!")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Импорт завершён!


In [44]:
# Количество точек (быстрый способ) --> точка (point) – это одна запись в векторной базе.
count = client.get_collection("articles").points_count
print(f"Сейчас в коллекции {count:,} точек")

# Дополнительная информация о коллекции
info = client.get_collection("articles")
print("Статус:", info.status)                         # green / yellow / red
print("Статус оптимизатора:", info.optimizer_status)  # ok / in_progress / ...
print("Сегментов:", info.segments_count)              # <- было segments_size (ошибка)
print("Всего векторов:", info.vectors_count)
print("Проиндексировано:", info.indexed_vectors_count)

Сейчас в коллекции 6 точек
Статус: green
Статус оптимизатора: ok
Сегментов: 4
Всего векторов: None
Проиндексировано: 0


**Лабораторная 1 — «Глубокая работа с одиночным узлом»**